# Extraction all data from the Mattilsynet API into raw data.

In [1]:
# Library imports
import requests
import time
import pickle
import os.path as path

API Dokumentasjon:
https://data.norge.no/datasets/288aa74c-e3d3-492e-9ede-e71503b3bfd9

In [2]:
# Generalisert funksjon for henting av all data.
# Må ha dato på string format 20032023 og ikke time.date. 
# Må gjøre endringer for dette.

def get_all_mattilsynet_reports(place = None, date = None):
    """Function that retrives raw data from the Mattilsynet API

    Args:
        place (string): City name e.g Oslo. Defaults to None.
        date (string): Date string e.g 20032023. Defaults to None.

    Returns:
        list: list of json objects
    """

    url = "https://hotell.difi.no/api/json/mattilsynet/smilefjes/tilsyn?"

    if place != None:
        url = url + "poststed=" + place
    if date != None:
        url = url + "&dato" + str(date)

    page_count = requests.get(url).json()['pages']
    list_of_all_reports = []
    
    for page_num in range(page_count):
        page_response = requests.get(url + "&page=" + str(page_num+1))
        list_of_all_reports.extend(page_response.json()['entries'])
        time.sleep(0.1)
    
    return list_of_all_reports   

In [3]:
# All the mattilsynet reports
list_of_all_reports = get_all_mattilsynet_reports(place = "Oslo")
len(list_of_all_reports)

5897

##### Data correction:
All orgnumbers should be 9 digits, there are two with out org numbers. These are retrived form the brønøysund website.

In [4]:
list_of_mising_org_num = ["927120186", "926532251"]
count = 0
for company in list_of_all_reports:
    if len(company['orgnummer']) != 9:
        company['orgnummer'] = list_of_mising_org_num[count]
        count +=1
        print(company)

{'tilsynsbesoektype': '0', 'poststed': 'OSLO', 'sakref': '2022/196963', 'tilsynsobjektid': 'Z2204220043365210127UTWSF_Tilsynsobjekt', 'orgnummer': '927120186', 'postnr': '0255', 'dato': '12092022', 'navn': 'Cartel´s Latin American Kitchen & Bar Solli Plass', 'tema1_no': 'Rutiner og ledelse', 'tema3_nn': 'Mat-handtering og tillaging', 'tema1_nn': 'Rutiner og leiing', 'tema3_no': 'Mat-håndtering og tilberedning', 'tilsynid': 'Z2209121220005640240CJRLL_TilsynAvtale', 'adrlinje1': 'Henrik Ibsens gate 90', 'karakter1': '0', 'adrlinje2': '', 'karakter2': '1', 'karakter3': '1', 'karakter4': '0', 'total_karakter': '1', 'tema4_no': 'Merking og sporbarhet', 'tema4_nn': 'Merking og sporbarheit', 'tema2_no': 'Lokaler og utstyr', 'status': '0', 'tema2_nn': 'Lokaler og utstyr'}
{'tilsynsbesoektype': '0', 'poststed': 'OSLO', 'sakref': '2023/2128', 'tilsynsobjektid': 'Z2212051327505721032OMQGR_Tilsynsobjekt', 'orgnummer': '926532251', 'postnr': '0159', 'dato': '04012023', 'navn': 'HAWKER OSLO AS', 'te

In [5]:
for company in list_of_all_reports:
    if len(company['orgnummer']) != 9:
        print(company)

##### All the org numbers are now accounted for

In [8]:
# Store the raw data as a pickle file.
# Gets the parent directory location enabeling storing the 
# data directly to the desired folder.
parent = path.abspath(path.join("mattilsynet_api.ipynb" ,"../../.."))
with open(f'{parent}/Data/Raw/mattilsynet_rawdata.obj', 'wb') as file:
	pickle.dump(list_of_all_reports, file)